Importing the libraries

In [1]:
import os
import cv2
from keras.models import load_model
import numpy as np
from pygame import mixer
import time # Timer
import sys  # Timer
import socket
from twilio.rest import Client
import datetime # using datetime module

pygame 2.0.1 (SDL 2.0.14, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


Initializing Pygame Mixer and assigning Alarm file to mixer 

In [2]:
mixer.init()
sound = mixer.Sound('Audio/alarm.mp3')

Initializing Timer variables to zero and creating object of Timer

In [3]:
start_time = 0

Alert sound function

In [4]:
def alert():
    mixer.init()
    sound = mixer.Sound('Audio/alarm.mp3')
    if(score>5 and score<10):
       
        # infinite loop till close
        # main.mainloop() 

        # Give an audio alert to the employee
        try:
            sound.play()
        except:  # isplaying = False
            pass

Setting Remote server to 1.1.1.1 and Creating Internet Conncetion Checker function

In [5]:
REMOTE_SERVER = "1.1.1.1"

def is_connected(hostname):
  try:
    host = socket.gethostbyname(hostname)
    s = socket.create_connection((host, 80), 2)
    s.close()
    return 'True'
  except:
     pass
  return 'False'

Creating a function for Setting File names for Video files recorded during drowsy state

In [6]:
def getAviNameWithDate(nameIn="Video.avi"):
    if not nameIn.endswith(".avi"):
        raise ValueError("filename must end on .avi")

    filename = nameIn.replace(".avi","_{0}.avi").format(datetime.datetime.now().strftime("%Y-%m-%d-%I-%M-%S"))

    if os.path.isfile(filename):             # if already exists
        fn2 = filename[0:-4]+'_{0}.avi'          # modify pattern to include a number
        count = 1
        while os.path.isfile(fn2.format(count)): # increase number until file not exists
            count += 1
            return fn2.format(count)                 # return file with number in it
    else:                                    # filename ok, return it
        return filename

Log file appending function

In [7]:
def logfile(start,end):
  f = open("log.txt","a+")
  f.write("\nDisconnected from {start} to {end}".format(start=start,end=end))
  f.close()

Setting file paths for cascade and Model files

In [8]:
face = cv2.CascadeClassifier('haar cascade files\haarcascade_frontalface_alt.xml')
leye = cv2.CascadeClassifier('haar cascade files\haarcascade_lefteye_2splits.xml')
reye = cv2.CascadeClassifier('haar cascade files\haarcascade_righteye_2splits.xml')
model = load_model('models/cnncat2.h5')

Setting Twilio Account sID and authentication token

In [9]:
account_sid = "AC305eed341fc053d5e854b401e15568b0"
auth_token = "e484047a314259614e1ce4f8349342a9"
client = Client(account_sid, auth_token)

Assigning all prediction related variables and fontstyle for label

In [10]:
path = os.getcwd()
cap = cv2.VideoCapture(0)

lbl=['Close','Open']
font = cv2.FONT_HERSHEY_COMPLEX_SMALL

count=0
score=0
thicc=2

rpred=[99]
lpred=[99]

In [ ]:
while(True):
    
    ret, frame = cap.read()
    height,width = frame.shape[:2]
    
#------ Initializing Timer variables to zero and creating object of Timer ------#
    time_start = time.time()
    seconds = 0
    minutes = 0
    iLog = 0

#------ Manipulating brightness and contrast of frame ------#
    # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # contrast = 3
    # brightness = 40
    # frame[:,:,2] = np.clip(contrast * frame[:,:,2] + brightness, 0, 255)
    # frame = cv2.cvtColor(frame, cv2.COLOR_HSV2BGR)
  
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(25,25))
    
    if(len(faces)==1):
    
        left_eye = leye.detectMultiScale(gray)
        right_eye =  reye.detectMultiScale(gray)

        cv2.rectangle(frame, (0,height-50) , (200,height) , (0,0,0) , thickness=cv2.FILLED )

        cv2.putText(frame,"Face is Detected",(10,20), font, 1,(255,255,255),1,cv2.LINE_AA)

        for (x,y,w,h) in faces:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(100,100,100),1)
        
        for (x,y,w,h) in left_eye:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,0),1)

        for (x,y,w,h) in right_eye:
            cv2.rectangle(frame,(x,y),(x,y),(0,0,0),1)

        for (x,y,w,h) in right_eye:
            r_eye=frame[y:y+h,x:x+w]
            count=count+1
            r_eye = cv2.cvtColor(r_eye,cv2.COLOR_BGR2GRAY)
            r_eye = cv2.resize(r_eye,(24,24))
            r_eye= r_eye/255
            r_eye=  r_eye.reshape(24,24,-1)
            r_eye = np.expand_dims(r_eye,axis=0)
            rpred = np.argmax(model.predict(r_eye), axis=-1)
            if(rpred[0]==1):
                lbl='Open' 
            if(rpred[0]==0):
                lbl='Closed'
            break

        for (x,y,w,h) in left_eye:
            l_eye=frame[y:y+h,x:x+w]
            count=count+1
            l_eye = cv2.cvtColor(l_eye,cv2.COLOR_BGR2GRAY)  
            l_eye = cv2.resize(l_eye,(24,24))
            l_eye= l_eye/255
            l_eye=l_eye.reshape(24,24,-1)
            l_eye = np.expand_dims(l_eye,axis=0)
            lpred = np.argmax(model.predict(l_eye), axis=-1)
            if(lpred[0]==1):
                lbl='Open'   
            if(lpred[0]==0):
                lbl='Closed'
            break
            
        if(rpred[0]==0 and lpred[0]==0):
            score=score+1
            cv2.putText(frame,"Closed",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
        else:
            score=0
            cv2.putText(frame,"Open",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
            
    ##----Score loops start from here------##
            
        if(score<0):
            score=0
    
        cv2.putText(frame,'Score:'+str(score),(100,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
            
        # if(score>10 and score<30):
        if(score==10):
            
            # Give an audio alert to the employee
            try:
                sound.play()
            except:  # isplaying = False
                pass
        
        if(score==15):  
            #Save image of employee in drowsy state
            cv2.imwrite(os.path.join(path,'proof.jpg'),frame)
            
            #Send a message alert to employer's phone
            message = client.messages \
            .create(body='Hello Niraj sir! Gaurav is sleeping!',from_='+18787288786',to='+919271909065')

        if(score==19):
            fourcc = cv2.VideoWriter_fourcc(*'XVID')
            out = cv2.VideoWriter(getAviNameWithDate(),fourcc, 5.0, (640,480))

        if(score>20 and score<150):
            #writes and saves videos
            out.write(frame)
            out.release()

    else:
        cv2.putText(frame,"Face is not Detected",(10,20), font, 1,(255,255,255),1,cv2.LINE_AA)

    # Invoke OpenCV window
    cv2.imshow('frame',frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    #------ Checking for internet connection throughout the run------#
    while is_connected(REMOTE_SERVER) == 'False':
        iLog = 1   # iLog has kept as 1 for reverting the loop of Log file appending

        if(minutes == 0 and seconds == 1):
            ct_start = datetime.datetime.now()  # ct_start stores the start time
            start_time = ct_start
            print(ct_start)

        # Timer to record the time interval of internet disconnection    
        try:
            sys.stdout.write("\r{minutes} Minutes {seconds} Seconds".format(minutes=minutes, seconds=seconds))
            sys.stdout.flush()
            time.sleep(1)
            seconds = int(time.time() - time_start) - minutes * 60
            if seconds >= 60:
                minutes += 1
                seconds = 0
        except KeyboardInterrupt:
            break

    if start_time!=0:
        if(iLog == 1):
            ct_end = datetime.datetime.now()   # ct_end stores the end time
            end_time = ct_end
            print(ct_end) 
            iLog+=1                 #increasing iLog more than 1 to break the if loop
            logfile(start_time,end_time)   #send the start and end time to logFile


Releasing CV2 Object and VideoCapture instance when job is finished

In [ ]:
cap.release()
cv2.destroyAllWindows()